### **Load Environment variables from .env file**

In [23]:
from dotenv import load_dotenv
import tiktoken
import openai
import pandas as pd
import os
import numpy as np
from openai.embeddings_utils import cosine_similarity, get_embedding
from IPython.display import display, HTML, JSON, Markdown

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

OPENAI_DAVINCI_DEPLOYMENT_NAME = os.getenv("OPENAI_DAVINCI_DEPLOYMENT_NAME")
OPENAI_DAVINCI_MODEL_NAME = os.getenv("OPENAI_DAVINCI_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

### **Read CSV using pandas**

In [4]:
input_datapath = "./data/recipes_onecol.csv"  
df = pd.read_csv(input_datapath)

In [5]:
df.head()

,Recipe
0,Recipe: Caramelized Bread Pudding with Chocola...
1,Recipe: White Sauce or Bechamel Sauce - Ingred...
2,"Recipe: Grilled Whole Mackerel with Lemon, Ore..."
3,Recipe: Christmas Croquembouche - Ingredients:...
4,Recipe: Chocolate Roll-Out Cookies - Ingredien...


### **Using OpenAI model ada for embeddings**

OpenAI offers one second-generation embedding model (denoted by -002 in the model ID) and 16 first-generation models (denoted by -001 in the model ID).
We recommend using text-embedding-ada-002 for nearly all use cases. 

#### Initialize Embeddings model 

In [6]:
# embedding model parameters
# encoding for text-embedding-ada-002
embedding_encoding = "cl100k_base"  
# the maximum for text-embedding-ada-002 is 8191
max_tokens = 8000 
# the number of reviews to embed
#in case of rate limit error, reduce the number of reviews
top_n = 150
encoding = tiktoken.get_encoding(embedding_encoding)

#### Count number of tokens in each row

In [15]:
# skip Recipes that are too long to embed > max_tokens
df["recipe_tokens"] = df.Recipe.apply(lambda x: len(encoding.encode(x)))
#df = df[df.recipe_tokens <= max_tokens].tail(top_n)
df = df[df.recipe_tokens <= max_tokens]

In [8]:
df.head()

,Recipe,recipe_tokens
0,Recipe: Caramelized Bread Pudding with Chocola...,746
1,Recipe: White Sauce or Bechamel Sauce - Ingred...,396
2,"Recipe: Grilled Whole Mackerel with Lemon, Ore...",749
3,Recipe: Christmas Croquembouche - Ingredients:...,1210
4,Recipe: Chocolate Roll-Out Cookies - Ingredien...,1047


#### Create embeddings and save on disk

In [10]:
# This may take a few minutes
df["embedding"] = df.Recipe.apply(lambda x: get_embedding(x, engine=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME))
# save the file
df.to_csv("./data/recipes_onecol_with_embeddings.csv")

#### Read embeddings

In [14]:
datafile_path = "./data/recipes_onecol_with_embeddings.csv"
df = pd.read_csv(datafile_path)

# convert the string representation of the embedding to a numpy array
df["embedding"] = df.embedding.apply(eval).apply(np.array)

#### Define a function to ask a question and get an answer from GPT-3.5

In [43]:
def ask_question(question, n=1):
    # get embedding for question
    question_embedding = get_embedding(
        question,
        engine=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME
    )

    # find the most similar embedding in the dataset
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, question_embedding))

    # sort the results by similarity
    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
    )
    
    # if we asked for more than one anwser (n=2 or more), we would return a list of answers
    return (results.similarity.iloc[0], ' \n##### '.join(results.Recipe.tolist()) )
    

#### Berries is similar to raspberries

In [56]:
answer = ask_question("Chocolate and berries")
display(Markdown( f"Similarity Score: {answer[0]}, {answer[1]}"))

Similarity Score: 0.8284768751105671, Recipe: Raspberry Sundaes with Chocolate Sauce and Roasted Almonds - Ingredients: 6 ounces bittersweet chocolate, finely chopped;3/4 cup water;2/3 cups sugar;2 tablespoons honey;3 tablespoons unsalted butter;Pinch of kosher salt;1 pint fresh raspberries (12 ounces);2 tablespoons sugar;2 tablespoons elderflower liqueur, such as St-Germain;2 tablespoons fresh lemon juice;2 pints vanilla ice cream;Chopped roasted unsalted almonds, for garnish- Instructions: To make the chocolate sauce, place a saucepan over medium-low heat, combine the chocolate, and water, vigorously stirring until completely melted and smooth. Let it come to a low simmer until small bubbles appear around the outside. Add sugar, honey and keep at a low simmer until thickened, about 3 minutes. Remove from heat and stir in butter and salt. Chocolate sauce will smooth out at is cools to room temperature. Store in an airtight container. In a bowl, gently toss together the raspberries, sugar, elderflower liqueur, and lemon juice. Let stand in the refrigerator for 30 minutes (or up to 2 hours) before serving. Assemble the sundae: Scoop ice cream into individual bowls. Top with chocolate sauce, almonds, and macerated raspberries.
To make the chocolate sauce, place a saucepan over medium-low heat, combine the chocolate, and water, vigorously stirring until completely melted and smooth. Let it come to a low simmer until small bubbles appear around the outside. Add sugar, honey and keep at a low simmer until thickened, about 3 minutes. Remove from heat and stir in butter and salt. Chocolate sauce will smooth out at is cools to room temperature. Store in an airtight container.
In a bowl, gently toss together the raspberries, sugar, elderflower liqueur, and lemon juice. Let stand in the refrigerator for 30 minutes (or up to 2 hours) before serving.
Assemble the sundae: Scoop ice cream into individual bowls. Top with chocolate sauce, almonds, and macerated raspberries.

#### Typo: Bukgur instead of Bulgur

In [55]:
answer = ask_question("Bukgur vegan recipe")
display(Markdown(f"Similarity Score: {answer[0]}, {answer[1]}"))

Similarity Score: 0.8508641994464585, Recipe: Bulgur Veggie Burgers with Lime Mayonnaise - Ingredients: 1/2 cup chopped onion, divided;1 tablespoon olive oil plus additional for brushing;1/2 cup bulgur;1 cup water;1 cup canned pinto beans, rinsed and drained;1 1/2 tablespoon soy sauce;3/4 cup walnuts (2 1/2 ounces);2 garlic cloves, coarsely chopped;1/2 cup packed cilantro sprigs;3/4 teaspoon ground cumin;1/4 teaspoon cayenne;1/4 cup mayonnaise;1/4 teaspoon grated lime zest;1/2 teaspoon fresh lime juice;4 slices multi-grain bread,toasted;Equipment: a perforated grill sheet;Accompaniments: lettuce; sliced tomato- Instructions: Cook half of onion with 1/4 teaspoon salt in oil in a small heavy saucepan over medium heat, stirring occasionally, until golden, 5 to 7 minutes. Add bulgur and water and cook, covered, over low heat until water is absorbed, 15 to 18 minutes. Transfer to a bowl and stir in beans and soy sauce. Pulse bulgur mixture, walnuts, garlic, cilantro, cumin, cayenne, a rounded 1/4 teaspoon salt, 1/2 teaspoon pepper, and remaining onion in a food processor until finely chopped. Form rounded 1/2 cups of mixture into 4 (31/2-inch-diameter) patties. Chill at least 10 minutes. While patties chill, stir together mayonnaise, zest, and juice. Prepare grill for direct-heat cooking over medium-hot charcoal (medium heat for gas). Put perforated grill sheet on grill and preheat 10 minutes. Brush patties all over with oil. Oil grill sheet, then grill burgers on grill sheet, covered only if using a gas grill, carefully turning once, until golden brown, about 8 minutes total. Serve burgers open-faced on toast with lime mayonnaise.
Cook half of onion with 1/4 teaspoon salt in oil in a small heavy saucepan over medium heat, stirring occasionally, until golden, 5 to 7 minutes. Add bulgur and water and cook, covered, over low heat until water is absorbed, 15 to 18 minutes. Transfer to a bowl and stir in beans and soy sauce.
Pulse bulgur mixture, walnuts, garlic, cilantro, cumin, cayenne, a rounded 1/4 teaspoon salt, 1/2 teaspoon pepper, and remaining onion in a food processor until finely chopped.
Form rounded 1/2 cups of mixture into 4 (31/2-inch-diameter) patties. Chill at least 10 minutes.
While patties chill, stir together mayonnaise, zest, and juice.
Prepare grill for direct-heat cooking over medium-hot charcoal (medium heat for gas). Put perforated grill sheet on grill and preheat 10 minutes.
Brush patties all over with oil.
Oil grill sheet, then grill burgers on grill sheet, covered only if using a gas grill, carefully turning once, until golden brown, about 8 minutes total.
Serve burgers open-faced on toast with lime mayonnaise.

In [50]:
answer = ask_question("Fish no alcohol")
display(Markdown(f"Similarity Score: {answer[0]}, {answer[1]}"))

Similarity Score: 0.7880176978357717, Recipe: Soft Fish Tacos - Ingredients: 1/4 cup mayonnaise;1/4 cup ketchup;1/4 cup crema mexicana*;1 cup all purpose flour;1 teaspoon fine sea salt;1/2 teaspoon ground pepper;1 cup dark beer, room temperature;13/4 pounds halibut, cut into 5x3/4-inch strips;1 lime, halved crosswise;12 white corn tortillas;Vegetable oil (for deep-frying);1 1/2 cups shredded red cabbage;2 large tomatoes, chopped;Lime wedges;Bottled hot pepper sauce- Instructions: For sauce: Mix all ingredients in bowl; season with salt and pepper.
Mix all ingredients in bowl; season with salt and pepper.
For batter and fish: Whisk flour, salt, and pepper in bowl; pour in beer, whisking until batter is smooth. Let stand 15 minutes. Sprinkle fish with salt and pepper. Squeeze some lime juice over each strip. Let stand 15 minutes. Mix fish into batter. Preheat oven to 200°F. Heat skillet over medium heat. Stack 2 tortillas. Sprinkle top with water. Place in skillet, wet side down. Heat 1 minute. Sprinkle top with water. Turn stack over; heat 1 minute. Transfer stack to large sheet of heavy-duty foil. Repeat. Enclose tortillas in foil. Place in oven. Pour oil into medium skillet to reach depth of 1 inch. Attach deep-fry thermometer; heat oil to 350°F. Slide 4 fish strips into oil. Fry until golden, about 4 minutes. Transfer fish to paper-towel-lined baking sheet; place in oven. Repeat. Fill each warm tortilla with 2 fish strips. Top with sauce, cabbage, tomato, squeeze of lime, and dash of hot pepper sauce.
Whisk flour, salt, and pepper in bowl; pour in beer, whisking until batter is smooth. Let stand 15 minutes.
Sprinkle fish with salt and pepper. Squeeze some lime juice over each strip. Let stand 15 minutes. Mix fish into batter.
Preheat oven to 200°F. Heat skillet over medium heat. Stack 2 tortillas. Sprinkle top with water. Place in skillet, wet side down. Heat 1 minute. Sprinkle top with water. Turn stack over; heat 1 minute. Transfer stack to large sheet of heavy-duty foil. Repeat. Enclose tortillas in foil. Place in oven.
Pour oil into medium skillet to reach depth of 1 inch. Attach deep-fry thermometer; heat oil to 350°F. Slide 4 fish strips into oil. Fry until golden, about 4 minutes. Transfer fish to paper-towel-lined baking sheet; place in oven. Repeat.
Fill each warm tortilla with 2 fish strips. Top with sauce, cabbage, tomato, squeeze of lime, and dash of hot pepper sauce.
*Cultured Mexican cream with a slightly nutty flavor and consistency of thin sour cream.
*Cultured Mexican cream with a slightly nutty flavor and consistency of thin sour cream.

In [51]:
answer = ask_question("Fish with wine")
display(Markdown(f"Similarity Score: {answer[0]}, {answer[1]}"))

Similarity Score: 0.8261893886663239, Recipe: Trout in Riesling - Ingredients: 2 tablespoons (1 oz) unsalted butter plus additional for greasing;1 medium carrot;1 celery rib;4 (6- to 8-oz) trout fillets with skin;1 1/2 teaspoons salt;1 teaspoon black pepper;4 medium shallots, cut lengthwise into 1/8-inch-thick slices (1 cup);2 fresh parsley sprigs;5 black peppercorns;1 1/2 cups water;1 1/2 cups dry Riesling;3/4 cup heavy cream;2 teaspoons cornstarch;2 teaspoons chopped fresh tarragon;1/2 teaspoon fresh lemon juice;Special equipment: an adjustable-blade slicer; tweezers or needlenose pliers- Instructions: Put oven rack in middle position and preheat oven to 375°F. Butter bottom and sides of a 13- by 9-inch glass baking dish. Cut carrot and celery lengthwise with slicer into very thin matchsticks (less than 1/8 inch wide and 2 1/2 inches long). Rinse trout and remove any pin bones with tweezers or needlenose pliers. Pat dry, then sprinkle 1/2 teaspoon each of salt and pepper on fish (flesh sides only). Arrange fillets in baking dish, skin sides down, overlapping slightly. Heat butter (2 tablespoons) in a 12-inch heavy skillet over moderate heat until foam subsides, then cook shallots, stirring, until softened, about 3 minutes (do not brown). Add carrot, celery, parsley, and peppercorns and cook, stirring, until vegetables are crisp-tender, about 5 minutes. Add water, wine, and 1/2 teaspoon salt and boil 1 minute. Remove skillet from heat and pour liquid over fish, then spread vegetables evenly in baking dish. Set skillet aside. Cover dish tightly with foil and bake until fish is just cooked through, 6 to 8 minutes. Transfer fish (topped with vegetables) to a platter and cover with foil to keep warm. Pour cooking liquid with any remaining vegetables through a sieve set over a bowl. Sprinkle vegetables from sieve over fish (discard peppercorns) and re-cover with foil. Spoon 1 cup of strained cooking liquid into cleaned skillet. Boil over high heat until reduced to about 1/2 cup, about 2 minutes. Whisk cream into cornstarch until smooth, then whisk into sauce and boil, whisking, until thickened, about 1 minute. Stir in tarragon, lemon juice, and remaining 1/2 teaspoon each of salt and pepper. Remove foil from fish and blot any liquid accumulated on platter with paper towels. Pour sauce over fish.
Put oven rack in middle position and preheat oven to 375°F. Butter bottom and sides of a 13- by 9-inch glass baking dish.
Cut carrot and celery lengthwise with slicer into very thin matchsticks (less than 1/8 inch wide and 2 1/2 inches long).
Rinse trout and remove any pin bones with tweezers or needlenose pliers. Pat dry, then sprinkle 1/2 teaspoon each of salt and pepper on fish (flesh sides only). Arrange fillets in baking dish, skin sides down, overlapping slightly.
Heat butter (2 tablespoons) in a 12-inch heavy skillet over moderate heat until foam subsides, then cook shallots, stirring, until softened, about 3 minutes (do not brown). Add carrot, celery, parsley, and peppercorns and cook, stirring, until vegetables are crisp-tender, about 5 minutes. Add water, wine, and 1/2 teaspoon salt and boil 1 minute. Remove skillet from heat and pour liquid over fish, then spread vegetables evenly in baking dish. Set skillet aside. Cover dish tightly with foil and bake until fish is just cooked through, 6 to 8 minutes.
Transfer fish (topped with vegetables) to a platter and cover with foil to keep warm. Pour cooking liquid with any remaining vegetables through a sieve set over a bowl. Sprinkle vegetables from sieve over fish (discard peppercorns) and re-cover with foil.
Spoon 1 cup of strained cooking liquid into cleaned skillet. Boil over high heat until reduced to about 1/2 cup, about 2 minutes. Whisk cream into cornstarch until smooth, then whisk into sauce and boil, whisking, until thickened, about 1 minute. Stir in tarragon, lemon juice, and remaining 1/2 teaspoon each of salt and pepper.
Remove foil from fish and blot any liquid accumulated on platter with paper towels. Pour sauce over fish.

In [53]:

answer = ask_question("Fish with alcohol")
display(Markdown(f"Similarity Score: {answer[0]}, {answer[1]}"))

Similarity Score: 0.8070105198549736, Recipe: Soft Fish Tacos - Ingredients: 1/4 cup mayonnaise;1/4 cup ketchup;1/4 cup crema mexicana*;1 cup all purpose flour;1 teaspoon fine sea salt;1/2 teaspoon ground pepper;1 cup dark beer, room temperature;13/4 pounds halibut, cut into 5x3/4-inch strips;1 lime, halved crosswise;12 white corn tortillas;Vegetable oil (for deep-frying);1 1/2 cups shredded red cabbage;2 large tomatoes, chopped;Lime wedges;Bottled hot pepper sauce- Instructions: For sauce: Mix all ingredients in bowl; season with salt and pepper.
Mix all ingredients in bowl; season with salt and pepper.
For batter and fish: Whisk flour, salt, and pepper in bowl; pour in beer, whisking until batter is smooth. Let stand 15 minutes. Sprinkle fish with salt and pepper. Squeeze some lime juice over each strip. Let stand 15 minutes. Mix fish into batter. Preheat oven to 200°F. Heat skillet over medium heat. Stack 2 tortillas. Sprinkle top with water. Place in skillet, wet side down. Heat 1 minute. Sprinkle top with water. Turn stack over; heat 1 minute. Transfer stack to large sheet of heavy-duty foil. Repeat. Enclose tortillas in foil. Place in oven. Pour oil into medium skillet to reach depth of 1 inch. Attach deep-fry thermometer; heat oil to 350°F. Slide 4 fish strips into oil. Fry until golden, about 4 minutes. Transfer fish to paper-towel-lined baking sheet; place in oven. Repeat. Fill each warm tortilla with 2 fish strips. Top with sauce, cabbage, tomato, squeeze of lime, and dash of hot pepper sauce.
Whisk flour, salt, and pepper in bowl; pour in beer, whisking until batter is smooth. Let stand 15 minutes.
Sprinkle fish with salt and pepper. Squeeze some lime juice over each strip. Let stand 15 minutes. Mix fish into batter.
Preheat oven to 200°F. Heat skillet over medium heat. Stack 2 tortillas. Sprinkle top with water. Place in skillet, wet side down. Heat 1 minute. Sprinkle top with water. Turn stack over; heat 1 minute. Transfer stack to large sheet of heavy-duty foil. Repeat. Enclose tortillas in foil. Place in oven.
Pour oil into medium skillet to reach depth of 1 inch. Attach deep-fry thermometer; heat oil to 350°F. Slide 4 fish strips into oil. Fry until golden, about 4 minutes. Transfer fish to paper-towel-lined baking sheet; place in oven. Repeat.
Fill each warm tortilla with 2 fish strips. Top with sauce, cabbage, tomato, squeeze of lime, and dash of hot pepper sauce.
*Cultured Mexican cream with a slightly nutty flavor and consistency of thin sour cream.
*Cultured Mexican cream with a slightly nutty flavor and consistency of thin sour cream.

In [52]:

answer = ask_question("Meat with alcohol")
display(Markdown(f"Similarity Score: {answer[0]}, {answer[1]}"))

Similarity Score: 0.8139065682641311, Recipe: Beer-Braised Holiday Top of the Rib - Ingredients: 1 tablespoon sweet or hot smoked paprika;1 tablespoon dark brown sugar;1 tablespoon ground cumin;1 tablespoon instant coffee granules;1 teaspoon kosher salt;1 teaspoon freshly ground black pepper;1/2 teaspoon onion powder;1/2 teaspoon garlic powder;One 5-pound top of the rib;3 tablespoons olive oil;Three 12-ounce cans or bottles dark lager beer;1 garlic head, halved horizontally;1 pound small parsnips, peeled and halved length wise;1 pound peeled baby carrots with greens- Instructions: 1. Preheat the oven to 325°F 2. Combine the paprika, brown sugar, cumin, coffee, salt, pepper, onion powder, and garlic powder in a small bowl and stir to mix. Pat the meat dry with paper towels and rub the spice mixture all over. 3. Heat the olive oil in a large Dutch oven over medium-high heat. Sear the meat until nicely browned, 4 to 6 minutes per side. Pour the beer over the meat, cover, and transfer to the oven. 4. Roast for 2 hours. Flip the meat over. Add the garlic, parsnips, and carrots. Cover and roast until the meat and vegetables are tender, about 1 1/2 hours more. Transfer the meat to a cutting board and let rest at least 15 minutes before slicing. Transfer the vegetable and garlic to a platter, reserving the pan sauce.
1. Preheat the oven to 325°F
2. Combine the paprika, brown sugar, cumin, coffee, salt, pepper, onion powder, and garlic powder in a small bowl and stir to mix. Pat the meat dry with paper towels and rub the spice mixture all over.
3. Heat the olive oil in a large Dutch oven over medium-high heat. Sear the meat until nicely browned, 4 to 6 minutes per side. Pour the beer over the meat, cover, and transfer to the oven.
4. Roast for 2 hours. Flip the meat over. Add the garlic, parsnips, and carrots. Cover and roast until the meat and vegetables are tender, about 1 1/2 hours more. Transfer the meat to a cutting board and let rest at least 15 minutes before slicing. Transfer the vegetable and garlic to a platter, reserving the pan sauce.